### Домашнее задание

#### Задание 1.
Скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('/Users/Adm1n/ml_for_business/Lesson4/data.csv')

In [4]:
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


#### Задание 2. 
Поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.¶

In [5]:
df = df.rename(columns = {'conversion': 'target', 
                          'offer': 'treatment'})

In [6]:
df.columns

Index(['recency', 'history', 'used_discount', 'used_bogo', 'zip_code',
       'is_referral', 'channel', 'treatment', 'target'],
      dtype='object')

In [7]:
features = (['recency', 'history', 'used_discount', 'used_bogo', 'zip_code',
       'is_referral', 'channel'])

#### Задание 3. 

Сделать разбиение набора данных на тренировочную и тестовую выборки


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size = 0.25, 
                                                    random_state = 42)

In [9]:
X_train.columns

Index(['recency', 'history', 'used_discount', 'used_bogo', 'zip_code',
       'is_referral', 'channel', 'treatment'],
      dtype='object')

In [10]:
X_train.columns == X_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True])

#### Задание 4. 
Сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

In [11]:
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


Проверим наличие пропусков

In [12]:
df.isnull().sum()

recency          0
history          0
used_discount    0
used_bogo        0
zip_code         0
is_referral      0
channel          0
treatment        0
target           0
dtype: int64

Изучим типы данных, обработаем категории

In [13]:
df.dtypes

recency            int64
history          float64
used_discount      int64
used_bogo          int64
zip_code          object
is_referral        int64
channel           object
treatment         object
target             int64
dtype: object

1) zip_code

In [14]:
df['zip_code'].nunique()

3

In [15]:
df['zip_code'].unique()

array(['Surburban', 'Rural', 'Urban'], dtype=object)

In [16]:
X_train['zip_code'].replace({'Urban': 1, 'Surburban': 2, 'Rural': 3}, inplace= True)
X_test['zip_code'].replace({'Urban': 1, 'Surburban': 2, 'Rural': 3}, inplace= True)

In [17]:
X_train['zip_code'].unique()

array([1, 3, 2], dtype=int64)


2) channel

In [18]:
df['channel'].unique()

array(['Phone', 'Web', 'Multichannel'], dtype=object)

In [19]:
X_train = pd.concat([X_train, pd.get_dummies(X_train['channel'], prefix='channel')], axis = 1)
X_test = pd.concat([X_test, pd.get_dummies(X_test['channel'], prefix = 'channel')],  axis = 1)

In [20]:
#Удалим исходный признак channel 
X_train.drop('channel', axis = 1, inplace=True)
X_test.drop('channel', axis = 1, inplace=True)

3) treatment

In [21]:
df['treatment'].unique()

array(['Buy One Get One', 'No Offer', 'Discount'], dtype=object)

Нам важно, было взаимодействие или нет. То есть значения Buy One Get и Discount можно объединить в общий флаг наличия взаимодействия

In [22]:
X_train['treatment'].replace({'Buy One Get One': 1, 'Discount': 1, 'No Offer': 0}, inplace= True)
X_test['treatment'].replace({'Buy One Get One': 1, 'Discount': 1, 'No Offer': 0}, inplace= True)

In [23]:
X_train['treatment'].unique()

array([0, 1], dtype=int64)

Отнормируем признак history и recency


In [24]:
X_train['recency'].unique()

array([ 6,  1,  7,  2, 10,  8,  9,  3, 11,  5, 12,  4], dtype=int64)

In [25]:
X_train['history'].unique()

array([ 92.45, 796.97, 390.14, ..., 462.53, 446.96, 787.51])

In [26]:
from sklearn.preprocessing import MinMaxScaler

In [27]:
scaler = MinMaxScaler()

In [28]:
X_train['history'] = scaler.fit_transform(X_train[['history']])
X_test['history'] = scaler.transform(X_test[['history']])

In [29]:
X_train['recency'] = scaler.fit_transform(X_train[['recency']])
X_test['recency'] = scaler.transform(X_test[['recency']])

In [30]:
X_train.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,treatment,channel_Multichannel,channel_Phone,channel_Web
7517,0.454545,0.019605,0,1,1,0,0,0,0,1
5343,0.000000,0.240736,0,1,3,1,1,1,0,0
35537,0.545455,0.113042,0,1,1,1,1,0,1,0


#### Задание 5. 
Провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями¶

In [31]:
pip install scikit-uplift==0.2.0

Note: you may need to restart the kernel to use updated packages.


In [32]:
from sklift.models import SoloModel, TwoModels, ClassTransformation
from sklift.metrics import uplift_at_k

from sklearn.ensemble import GradientBoostingClassifier

In [33]:
models_results = {
    'models': [],
    'uplift@10%': [],
    'uplift@20%': [],
    'uplift@30%': []
}

In [34]:
treat_train = X_train['treatment']
treat_test = X_test['treatment']

X_train = X_train.drop('treatment', axis = 1)
X_test = X_test.drop('treatment', axis = 1)

1) Одна модель с признаком коммуникации (S learner/SoloModel)

In [35]:
sm = SoloModel(GradientBoostingClassifier(n_estimators=40, max_depth = 3, random_state = 1))

sm.fit(X_train, y_train, treatment=treat_train)

uplift_sm = sm.predict(X_test)

sm_score_10 = uplift_at_k(y_test, uplift=uplift_sm, treatment = treat_test, strategy='by_group', k=0.1)
sm_score_20 = uplift_at_k(y_test, uplift=uplift_sm, treatment = treat_test, strategy='by_group', k=0.2)
sm_score_30 = uplift_at_k(y_test, uplift=uplift_sm, treatment = treat_test, strategy='by_group', k=0.3)
print(f'uplift@10%: {sm_score_10:.4f}, uplift@20%: {sm_score_20:.4f}, uplift@30%: {sm_score_30}')

models_results['models'].append('SoloModel')
models_results['uplift@10%'].append(sm_score_10)
models_results['uplift@20%'].append(sm_score_20)
models_results['uplift@30%'].append(sm_score_20)

uplift@10%: 0.1512, uplift@20%: 0.1186, uplift@30%: 0.10487908598091827


2) Модель с транформацией таргета

In [36]:
ct = ClassTransformation(GradientBoostingClassifier(n_estimators=40, max_depth = 3, random_state = 1))

ct.fit(X_train, y_train, treatment=treat_train)

uplift_ct = ct.predict(X_test)

ct_score_10 = uplift_at_k(y_test, uplift=uplift_ct, treatment = treat_test, strategy='by_group', k=0.1)
ct_score_20 = uplift_at_k(y_test, uplift=uplift_ct, treatment = treat_test, strategy='by_group', k=0.2)
ct_score_30 = uplift_at_k(y_test, uplift=uplift_ct, treatment = treat_test, strategy='by_group', k=0.3)
print(f'uplift@10%: {ct_score_10:.4f}, uplift@20%: {ct_score_20:.4f}, uplift@30%: {ct_score_30}')

models_results['models'].append('ClassTransformation')
models_results['uplift@10%'].append(ct_score_10)
models_results['uplift@20%'].append(ct_score_20)
models_results['uplift@30%'].append(ct_score_20)

<ipython-input-36-6424ce9f2bdb>:3: UserWarning: It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.
  ct.fit(X_train, y_train, treatment=treat_train)


uplift@10%: 0.1456, uplift@20%: 0.1018, uplift@30%: 0.09244299572602055


3) Вариант с двумя независимыми моделями

In [37]:
tm = TwoModels(estimator_trmnt = GradientBoostingClassifier(n_estimators=40, max_depth = 3, random_state = 1),
               estimator_ctrl =  GradientBoostingClassifier(n_estimators=40, max_depth = 3, random_state = 1),
               method='vanilla') 
#vanilla - независимые модели
#ddr_control или ddr_treatment - зависимые модели

tm.fit(X_train, y_train, treatment=treat_train)

uplift_tm = tm.predict(X_test)

tm_score_10 = uplift_at_k(y_test, uplift=uplift_tm, treatment = treat_test, strategy='by_group', k=0.1)
tm_score_20 = uplift_at_k(y_test, uplift=uplift_tm, treatment = treat_test, strategy='by_group', k=0.2)
tm_score_30 = uplift_at_k(y_test, uplift=uplift_tm, treatment = treat_test, strategy='by_group', k=0.3)
print(f'uplift@10%: {tm_score_10:.4f}, uplift@20%: {tm_score_20:.4f}, uplift@30%: {tm_score_30}')

models_results['models'].append('TwoModels')
models_results['uplift@10%'].append(tm_score_10)
models_results['uplift@20%'].append(tm_score_20)
models_results['uplift@30%'].append(tm_score_20)

uplift@10%: 0.1259, uplift@20%: 0.1040, uplift@30%: 0.09639764589758444


#### Задание 6. 
В конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей


In [38]:
evaluate_models = pd.DataFrame(models_results)
evaluate_models

,models,uplift@10%,uplift@20%,uplift@30%
0,SoloModel,0.151230,0.118606,0.118606
1,ClassTransformation,0.145643,0.101754,0.101754
2,TwoModels,0.125875,0.103992,0.103992


#### Вывод: 
Самые высокие показатели показала модель SoloModel. При одинаковых гиперпараметрах GradientBoosting эта модель дала результаты заметно лучше остальных.
В процессе подбора гиперпараметров лучшие значения среди моделей иногда давала ClassTransformation

#### Задание 7. 
Построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево

не устанавливается пакет(